# Get image embeddings from Azure Computer Vision

In [13]:
# imports
import requests
from tenacity import retry, stop_after_attempt, wait_random_exponential
import os
from dotenv import load_dotenv
import pandas as pd

In [9]:
# Load envs
load_dotenv('.env')

# Azure Computer Vision
key = os.getenv("azure_cv_key")
endpoint = os.getenv("azure_cv_endpoint")

# Images to process
image_dir = './val2017/'

In [8]:
@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6)) # automatic retry in case of a failing API call
def get_embedding(imagefile):
    """
    Get embedding from an image using Azure Computer Vision 4
    """
    # settings
    model = "?api-version=2023-02-01-preview&modelVersion=latest"
    url = endpoint + "/computervision/retrieval:vectorizeImage" + model
    headers = {
        "Content-type": "application/octet-stream",
        "Ocp-Apim-Subscription-Key": key,
    }

    # Read the image file
    with open(imagefile, "rb") as f:
        data = f.read()

    # Sending the requests
    r = requests.post(url, data=data, headers=headers)
    results = r.json()
    embeddings = results['vector']

    return embeddings

In [ ]:
df = pd.DataFrame()
num_files = len(os.listdir(image_dir))
for i, filename in enumerate(os.listdir(image_dir)):
    print(f"{i} out of {num_files} - {filename}")
    embedding = get_embedding(image_dir + filename)
    row = pd.DataFrame({"filename": filename, "embedding": [embedding]})
    df = pd.concat([df, row], axis=0)

In [2]:
df.sample(20)

,filename,embedding
0,000000153299.jpg,"[-4.0234375, 2.1035156, -3.0058594, -2.7753906..."
0,000000168337.jpg,"[-2.8046875, 3.3261719, 0.8041992, -0.42211914..."
0,000000347664.jpg,"[-1.4189453, -1.4560547, 1.7724609, 1.2861328,..."
0,000000255401.jpg,"[-0.13977051, 0.40551758, -2.4082031, -0.05279..."
0,000000018833.jpg,"[-0.69189453, -0.18640137, -3.3378906, 0.15905..."
0,000000311518.jpg,"[-0.6010742, 2.609375, 5.234375, -0.28808594, ..."
0,000000288042.jpg,"[-3.1171875, 2.1113281, 2.4179688, 3.0644531, ..."
0,000000520264.jpg,"[-2.4902344, 0.14440918, 1.4882812, -1.4208984..."
0,000000013774.jpg,"[3.6230469, 0.796875, 3.7246094, 0.42529297, -..."
0,000000377497.jpg,"[-1.140625, -0.32373047, -2.5703125, -1.544921..."


In [10]:
# Embeddings shape
df.iloc[0]['embedding'].shape

(1024,)

In [58]:
# Save results
df.to_parquet("azurecv_image_embeddings.parquet")